In [1]:
%load_ext autoreload
%autoreload 2
import os
import random
import sqlite3 
import numpy as np
import json
import math
from tqdm.notebook import tqdm
from tqdm import tqdm
tqdm.pandas()

import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

import Advan_operator as ad_op  

pd.set_option('display.max_columns', 20)
import calendar

In [2]:
data_path = r'E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\Wild_fire\adjusted_negative_hourly_population'
save_path = r'E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\Wild_fire\adjusted_negative_hourly_population' 

years = [2022]
months = list(range(1, 13))

landscan_daytime_fname =   r"E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\Wild_fire\hourly_map_test\Landscan_daytime_2021_CBG.csv"
landscan_nighttime_fname = r"E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\Wild_fire\hourly_map_test\Landscan_nighttime_2021_CBG.csv"

# hourly_popu_fname = fr"D:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\Wild_fire\hourly_map_test_2024_home_panel_dell_add_stop_factor\CBG_population_hourly_{year}{month:02}.csv"
ACS_file = r"E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_data\Safegraph_bias\cbg_acs_2019_county_tract_new20230929_cleaned.csv"
CBG_place_fname = r'E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\Wild_fire\vectors\CBG_place.gpkg'

# desktop 2018
landscan_fname = r"E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\Wild_fire\hourly_map_test\Landscan_daytime_2021_CBG.csv"
# hourly_popu_fname = fr"E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\Wild_fire\hourly_map_test_2024_home_panel_dell_add_stop_factor\CBG_population_hourly_{year}{month:02}.csv"
# hourly_popu_fname = fr"E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\Wild_fire\adjusted_negative_hourly_population\CBG_population_hourly_{year}{month:02}.csv"
# hourly_popu_fname = fr"E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\Wild_fire\hourly_map_test_2024_home_panel_dell_add_stop_factor\CBG_population_hourly_{year}{month:02}.csv"

landscan_day_df = pd.read_csv(landscan_daytime_fname, dtype={"GEOID":str}, usecols=['GEOID', 'SUM']).rename(columns={"SUM": "landscan_day", "GEOID":"CBG"}).set_index("CBG")
landscan_night_df = pd.read_csv(landscan_nighttime_fname, dtype={"GEOID":str}, usecols=['GEOID', 'SUM']).rename(columns={"SUM": "landscan_night", "GEOID":"CBG"}).set_index("CBG")

ACS_df = pd.read_csv(ACS_file, dtype={'fips':str}).iloc[:, :2].rename(columns={"fips": "CBG"}).set_index("CBG").astype(int)
ACS_df = ACS_df.merge(landscan_day_df, left_index=True, right_index=True).merge(landscan_night_df, left_index=True, right_index=True)
ACS_df



totalpopulation  landscan_day  landscan_night
CBG                                                        
010010201001              730         392.0           580.0
010010201002             1263         436.0          1194.0
010010202001              835        1622.0           904.0
010010202002             1124        2566.0          1116.0
010010203001             2774         974.0          2254.0
...                       ...           ...             ...
560459511001             1450         951.0          1408.0
560459511002             1844         956.0          1798.0
560459513001             1314         803.0          1041.0
560459513002             1164        1489.0          1179.0
560459513003             1277        1255.0          1319.0

[216327 rows x 3 columns]

In [3]:

landscan_day_df

landscan_day
CBG                       
010010201001         392.0
010010201002         436.0
010010202001        1622.0
010010202002        2566.0
010010203001         974.0
...                    ...
560459511001         951.0
560459511002         956.0
560459513001         803.0
560459513002        1489.0
560459513003        1255.0

[216327 rows x 1 columns]

In [4]:
CBG_place_gdf = gpd.read_file(CBG_place_fname)

In [5]:
df_list = []
for year in years:
    for month in months:
        hourly_popu_fname = fr"E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\Wild_fire\adjusted_negative_hourly_population\CBG_population_hourly_{year}{month:02}.csv"
        print(hourly_popu_fname)
        df = pd.read_csv(hourly_popu_fname, dtype={'CBG':str}).set_index('CBG').astype(np.int16)
        df_list.append(df)
        
all_df = pd.concat(df_list, axis=1)
df_list = []
# all_df

E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\Wild_fire\adjusted_negative_hourly_population\CBG_population_hourly_202201.csv
E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\Wild_fire\adjusted_negative_hourly_population\CBG_population_hourly_202202.csv
E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\Wild_fire\adjusted_negative_hourly_population\CBG_population_hourly_202203.csv
E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\Wild_fire\adjusted_negative_hourly_population\CBG_population_hourly_202204.csv
E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\Wild_fire\adjusted_negative_hourly_population\CBG_population_hourly_202205.csv
E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\Wild_fire\adjusted_negative_hourly_population\CBG_population_hourly_202206.csv
E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\Wild_fir

In [6]:
hourly_index = pd.date_range(start='2022-01-01', end='2022-12-31 23:00:00', freq='H')
all_df.index = all_df.index.str.zfill(12)
all_df.columns = hourly_index 
all_df

2022-01-01 00:00:00  2022-01-01 01:00:00  2022-01-01 02:00:00  \
CBG                                                                           
010010201001                909.0                106.0                273.0   
010010201002               3000.0               2302.0               2876.0   
010010202001               1117.0                366.0                511.0   
010010202002               1654.0               1055.0               1082.0   
010010203001               3021.0               3132.0               3687.0   
...                           ...                  ...                  ...   
721279801051                  NaN                  NaN                  NaN   
040137233031                  NaN                  NaN                  NaN   
061150409022                  NaN                  NaN                  NaN   
720339800041                  NaN                  NaN                  NaN   
720871105002                  NaN                  NaN                  NaN   

              2022-01-01 03:00:00  2022-01-01 04:00:00  2022-01-01 05:00:00  \
CBG                                                                           
010010201001                434.0                553.0                610.0   
010010201002               2786.0               3085.0               3082.0   
010010202001                650.0                690.0                802.0   
010010202002                854.0                819.0               1140.0   
010010203001               4594.0               3525.0               3046.0   
...                           ...                  ...                  ...   
721279801051                  NaN                  NaN                  NaN   
040137233031                  NaN                  NaN                  NaN   
061150409022                  NaN                  NaN                  NaN   
720339800041                  NaN                  NaN                  NaN   
720871105002                  NaN                  NaN                  NaN   

              2022-01-01 06:00:00  2022-01-01 07:00:00  2022-01-01 08:00:00  \
CBG                                                                           
010010201001                539.0                613.0                431.0   
010010201002               2178.0               1781.0               1066.0   
010010202001                670.0                688.0                647.0   
010010202002                943.0               1250.0               1654.0   
010010203001               2950.0               2827.0               2572.0   
...                           ...                  ...                  ...   
721279801051                  NaN                  NaN                  NaN   
040137233031                  NaN                  NaN                  NaN   
061150409022                  NaN                  NaN                  NaN   
720339800041                  NaN                  NaN                  NaN   
720871105002                  NaN                  NaN                  NaN   

              2022-01-01 09:00:00  ...  2022-12-31 14:00:00  \
CBG                                ...                        
010010201001                408.0  ...                -41.0   
010010201002                763.0  ...                 65.0   
010010202001                342.0  ...                 83.0   
010010202002               1079.0  ...                 37.0   
010010203001               2429.0  ...               -395.0   
...                           ...  ...                  ...   
721279801051                  NaN  ...                627.0   
040137233031                  NaN  ...                878.0   
061150409022                  NaN  ...                  7.0   
720339800041                  NaN  ...                  0.0   
720871105002                  NaN  ...                602.0   

              2022-12-31 15:00:00  2022-12-31 16:00:00  2022-12-31 17:00:00  \
CBG                            

In [7]:

# all_df

In [8]:


# CBG_place_gdf

In [7]:
CBG_place_hourly_gdf = CBG_place_gdf.merge(ACS_df, right_index=True,  left_on='CBG').merge(all_df, right_index=True, left_on='CBG')
CBG_place_hourly_gdf
# CBG_place_hourly_gdf

sub_area_ratio           CBG    place              NAME  \
0             0.991480  010730059033  0107000        Birmingham   
1             0.953754  010730059031  0107000        Birmingham   
2             0.945075  010730059053  0107000        Birmingham   
3             1.000000  010730059051  0107000        Birmingham   
4             1.000000  010730057023  0107000        Birmingham   
...                ...           ...      ...               ...   
146448        0.989965  560210004012  5671800     South Greeley   
146449        0.962990  560210004013  5629300  Fox Farm-College   
146450        0.899414  560210004022  5629300  Fox Farm-College   
146451        1.000000  560319594006  5683040         Wheatland   
146452        0.998854  560319594005  5683040         Wheatland   

                    NAMELSAD STUSPS STATE_NAME  \
0            Birmingham city     AL    Alabama   
1            Birmingham city     AL    Alabama   
2            Birmingham city     AL    Alabama   
3            Birmingham city     AL    Alabama   
4            Birmingham city     AL    Alabama   
...                      ...    ...        ...   
146448     South Greeley CDP     WY    Wyoming   
146449  Fox Farm-College CDP     WY    Wyoming   
146450  Fox Farm-College CDP     WY    Wyoming   
146451        Wheatland town     WY    Wyoming   
146452        Wheatland town     WY    Wyoming   

                                                 geometry  totalpopulation  \
0       MULTIPOLYGON (((-86.69174 33.60098, -86.69159 ...             1991   
1       MULTIPOLYGON (((-86.68297 33.59476, -86.68294 ...             2187   
2       MULTIPOLYGON (((-86.70047 33.61792, -86.70042 ...             2741   
3       MULTIPOLYGON (((-86.71176 33.59397, -86.71124 ...             1213   
4       MULTIPOLYGON (((-86.88307 33.47244, -86.88301 ...              412   
...                                                   ...              ...   
146448  MULTIPOLYGON (((-104.82115 41.08882, -104.8208...              597   
146449  MULTIPOLYGON (((-104.78868 41.10225, -104.7884...              680   
146450  MULTIPOLYGON (((-104.80266 41.11944, -104.8038...             2884   
146451  MULTIPOLYGON (((-104.96564 42.05622, -104.9655...              842   
146452  MULTIPOLYGON (((-104.98257 42.05634, -104.9825...             1294   

        landscan_day  ...  2022-12-31 14:00:00  2022-12-31 15:00:00  \
0             1060.0  ...                199.0                199.0   
1             1508.0  ...                218.0                218.0   
2             1134.0  ...               -240.0               1280.0   
3             1250.0  ...               1020.0               1696.0   
4              234.0  ...                581.0                910.0   
...              ...  ...                  ...                  ...   
146448         691.0  ...                 29.0                 59.0   
146449         282.0  ...                 68.0                 68.0   
146450        1222.0  ...                288.0                288.0   
146451        1540.0  ...               1326.0                133.0   
146452        1206.0  ...               -127.0                723.0   

        2022-12-31 16:00:00  2022-12-31 17:00:00  2022-12-31 18:00:00  \
0                     199.0               -105.0                324.0   
1                     218.0                218.0                 76.0   
2                    1445.0               2761.0               2078.0   
3                    1316.0               1125.0               2195.0   
4                    1345.0               1736.0               1281.0   
...                     ...                  ...                  ...   
146448                 59.0                 59.0                 59.0   
146449                 68.0                 68.0                 68.0   
146450                288.0                288.0                288.0   
146451                392.0               1680.0               1401.0   
146452   

In [10]:
# CBG_place_hourly_gdf 

In [11]:
# CBG_place_hourly_gdf
# saved_fname = r'E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\Wild_fire\CBG_place_2022_population.csv'

# CBG_place_hourly_gdf.drop(columns=['geometry']).to_csv(saved_fname, index=False)

In [12]:
# saved_fname = r'E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\Wild_fire\CBG_place_2022_population.gpkg'
# CBG_place_hourly_gdf.to_file(saved_fname, driver="GPKG")

In [8]:
CBG_place_hourly_gdf['NAMELSAD'].nunique()

10401

In [9]:
df = CBG_place_hourly_gdf.query('NAME == "Myrtle Beach" ')
# df = CBG_place_hourly_gdf.query('NAME == "Ocean City" and STUSPS == "MD" ')
# df = CBG_place_hourly_gdf[CBG_place_hourly_gdf['NAMELSAD'].str.contains('Miami Beach city')]
# df = CBG_place_hourly_gdf.query('NAMELSAD == "Miami Beach city" ')
# df = CBG_place_hourly_gdf[CBG_place_hourly_gdf['NAMELSAD'].str.contains('Daytona Beach city')]
# df = CBG_place_hourly_gdf[CBG_place_hourly_gdf['NAMELSAD'].str.contains('Port Aransas')]
# df = CBG_place_hourly_gdf[CBG_place_hourly_gdf['NAMELSAD'].str.contains('Capitola')]
# df = CBG_place_hourly_gdf[CBG_place_hourly_gdf['NAMELSAD'].str.contains('St. Augustine')]
# df = CBG_place_hourly_gdf[CBG_place_hourly_gdf['NAMELSAD'].str.contains('Cape May')]
# df = CBG_place_hourly_gdf[CBG_place_hourly_gdf['NAMELSAD'].str.contains('San Clemente')]
# df = CBG_place_hourly_gdf[CBG_place_hourly_gdf['NAMELSAD'].str.contains('Newport')].query('STATE_NAME == "Rhode Island" ')
# df = CBG_place_hourly_gdf[CBG_place_hourly_gdf['NAMELSAD'].str.contains('Solana Beach')] 
# df = CBG_place_hourly_gdf[CBG_place_hourly_gdf['NAMELSAD'].str.contains('Provincetown')]#.query('STATE_NAME == "Rhode Island" ')
# df = CBG_place_hourly_gdf.query('NAME == "State College" ')
# df = CBG_place_hourly_gdf.query('NAME == "Columbia" ').query('STUSPS == "SC" ')
# df = CBG_place_hourly_gdf[CBG_place_hourly_gdf['NAMELSAD'].str.contains('Holden Beach town')]
# df = CBG_place_hourly_gdf[CBG_place_hourly_gdf['CBG'].str.startswith('36061')]

df

MemoryError: Unable to allocate 9.48 GiB for an array with shape (8763, 145220) and data type float64

In [ ]:
df.iloc[:, 8:]

In [ ]:
sum_series = df.iloc[:, 8:].sum()

ACS_popu = sum_series.iloc[0]
LandScan_day_popu = sum_series.iloc[1]
LandScan_night_popu = sum_series.iloc[2]

place_name = '_'.join(df[['NAMELSAD', 'STUSPS', 'place']].iloc[0].to_list())

mini_popu_ratio = 0.1
sum_series = sum_series.mask(sum_series < 0, ACS_popu * mini_popu_ratio)

In [ ]:
df['mini_popu'] = df['totalpopulation'] * 0.1
df

In [ ]:
hourly_popu_df = df.iloc[:, 11:]
hourly_popu_df = hourly_popu_df.apply(lambda x: x.where(x >= 0, df['mini_popu']))
df = pd.concat([df.iloc[:, :11], hourly_popu_df.iloc[:, :-1]], axis=1)

In [ ]:
import matplotlib.pyplot as plt

def plot_population(df):
    fig, ax = plt.subplots(figsize = (25, 5))
    sum_series = df.iloc[:, 8:].sum()

    ACS_popu = sum_series.iloc[0]
    LandScan_day_popu = sum_series.iloc[1]
    LandScan_night_popu = sum_series.iloc[2]

    place_name = '_'.join(df[['NAMELSAD', 'STUSPS', 'place']].iloc[0].to_list())

    mini_popu_ratio = 0.1
    sum_series = sum_series.mask(sum_series < 0, ACS_popu * mini_popu_ratio)

    plt.plot(sum_series.iloc[3:], label="Hourly", alpha=0.6, linewidth=1.2)
    plt.plot(sum_series.iloc[3:].rolling(window=24, min_periods=1).mean(), label="Daily (mean of 24-hour)", color='blue', alpha=0.6)  # rolling 24 hours

    plt.axhline(y=ACS_popu, color = 'green', linestyle = '-', label="ACS population") 
    plt.axhline(y=LandScan_day_popu, color = 'orange', linestyle = '-', label="LandScan Daytime") 
    plt.axhline(y=LandScan_night_popu, color = 'black', linestyle = '-', label="LandScan Nighttime") 

    # grey weekends
    hourly_index = pd.date_range(start='2022-01-01', end='2022-12-31 23:00:00', freq='H')
    for start in hourly_index[hourly_index.weekday >= 5]:
        plt.axvspan(start, start + pd.Timedelta(days=1), color='gainsboro', alpha=0.02)

    saved_fname = os.path.join(save_path, 'place_plots',   f"{place_name}_{year}.png")  
    plt.title(df.iloc[0]['NAMELSAD'] + ", " + df.iloc[0]['STATE_NAME'], fontsize=18)
    plt.ylabel('Population')
    plt.legend(title='Population')
    # plt.savefig(saved_fname, dpi=300)
    # plt.close()
    plt.show()
 
plot_population(df)

In [89]:
STOP

NameError: name 'STOP' is not defined

## Show the map

In [ ]:
CBG_2019_fname = r"E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\Wild_fire\blockgoups2019.zip"

In [ ]:
CBG2019_gdf = gpd.read_file(CBG_2019_fname)
CBG2019_gdf['county_FIPS'] = CBG2019_gdf['GEOID'].astype(str).str.zfill(12).str[:5]
CBG2019_gdf['CBG'] = CBG2019_gdf['GEOID'].astype(str).str.zfill(12)


In [ ]:
CBG2019_gdf

In [379]:
year = 2022
month = 4
day = 9

hourly_index = pd.date_range(start=f'2022-{month:02}-{day:02}', end=f'2022-{month:02}-{day:02} 23:00:00', freq='H')
df = df.set_index('CBG')[hourly_index]
vmax = df.mean(axis=1).mean() + df.mean(axis=1).std() * 3
print("vmax:", vmax)
df

vmax: 17716.18901565814


2022-04-09 00:00:00  2022-04-09 01:00:00  2022-04-09 02:00:00  \
CBG                                                                           
450510503032               9649.0               6374.0               2915.0   
450510506002               9928.0               7264.0               4560.0   
450510507001               8294.0               4522.0               3647.0   
450510504011               1879.0               1431.0               1325.0   
450510504022               4193.0               3500.0               2234.0   
450510505001               1174.0               1455.0               1533.0   
450510505002               4982.0               2400.0               2473.0   
450510505003               6996.0               3346.0               2847.0   
450510506001               5368.0               4198.0               4212.0   
450510509003               2045.0               2433.0               2370.0   
450510510001                913.0               1158.0               1303.0   
450510503031               2715.0               2058.0               1910.0   
450519801001                831.0                386.0                466.0   
450510504021               2125.0               1801.0               1724.0   
450510509001               2367.0               1175.0               1334.0   
450510509002               4945.0               2986.0               2570.0   
450510517001              19784.0              10305.0              10326.0   
450510501022               3362.0               1859.0               1683.0   
450510504012               1404.0               1208.0               1277.0   
450510510002               3925.0               3944.0               3994.0   

              2022-04-09 03:00:00  2022-04-09 04:00:00  2022-04-09 05:00:00  \
CBG                                                                           
450510503032               2596.0               1857.0               2107.0   
450510506002               3618.0               2484.0               2738.0   
450510507001               2069.0               2310.0               1829.0   
450510504011               1601.0               1373.0               1388.0   
450510504022               1840.0               1205.0                845.0   
450510505001               1341.0               1526.0               1374.0   
450510505002               1486.0               1643.0               1329.0   
450510505003               2176.0               2356.0               2684.0   
450510506001               3572.0               3574.0               3571.0   
450510509003               2696.0               2372.0               2900.0   
450510510001               1452.0               1413.0               1428.0   
450510503031               1630.0               1785.0               2551.0   
450519801001                204.0               1622.0               1214.0   
450510504021               1512.0               1478.0               1420.0   
450510509001               1430.0               1584.0               1659.0   
450510509002               3683.0               5363.0               3503.0   
450510517001               3654.0               3409.0               2749.0   
450510501022               1513.0               1603.0               2074.0   
450510504012               1469.0               1374.0               1390.0   
450510510002               4271.0               4145.0               4151.0   

              2022-04-09 06:00:00  2022-04-09 07:00:00  2022-04-09 08:00:00  \
CBG                                                                           
450510503032               3355.0               5893.0               8432.0   
450510506002               2922.0               5415.0               7078.0   
450510507001               2417.0               2617.0               3640.0   
450510504011               1296.0               1602.0               2692.0   
450510504022                734.0                698.0        

In [309]:
# df.index.astype(str).zfill(5

In [ ]:
merged_gdf = CBG2019_gdf.merge(df, left_on='CBG', right_index=True)
merged_gdf.plot()

In [ ]:
vmax = df.mean(axis=1).mean() + df.mean(axis=1).std() * 3
vmax
merged_gdf#.iloc[:, hour_column]

In [ ]:
place_name = "Manhattan, NY"
place_name = "State College, PA"
place_name = 'Myrtle Beach, SC'
place_name = 'Columbia, SC'
place_name = 'Ocean City, MD'

row_cnt = 3
col_cnt = 9


fig = plt.figure(figsize=(20, 7))
ax_idx = 0



cmap = 'viridis'
hour_column = 0

for row in range(1, row_cnt + 1):    
    for col in range(1, col_cnt + 1):   
        ax_idx += 1
        if col == (col_cnt):
            # print("skip: col = ", col)
            continue                  
        # print("row, col, ax_idx, hour_column:", row, col, ax_idx, hour_column)
        
        ax=fig.add_subplot(row_cnt, col_cnt, ax_idx)
        ax.axis('off')
        ax.set_title(f"{hour_column:02}:00:00")
        
        merged_gdf.plot(column=hourly_index[hour_column],  ax=ax, vmax=vmax, cmap=cmap) 
        
        hour_column = hour_column + 1
        
        if hour_column == 24:
            break
    if hour_column == 24:
        break    

ACS_LandScan_gdf = merged_gdf[['CBG', 'geometry']].merge(ACS_df.loc[df.index], left_on='CBG', right_index=True)

ax = fig.add_subplot(2, col_cnt,  9)
ax.axis('off')
ax.set_title(f"ACS 2019 Population")
ACS_popu_df = ACS_LandScan_gdf.plot(column='totalpopulation',  ax=ax, vmax=vmax, cmap=cmap)
new_position = [0.87, 0.55, 0.12, 0.33]  # [left, bottom, width, height]
ax.set_position(new_position)


# hourly population
ax = fig.add_subplot(2, col_cnt,  18)
# ax.axis('off')
hourly_popu_list = df.sum().to_list()
ax.plot(hourly_popu_list, label='hourly population')
ax.axhline(ACS_LandScan_gdf['totalpopulation'].sum(), color='red', label='ACS population')
ax.set_title('Hourly population')
ax.legend()
# ax3.set_aspect('auto')
new_position = [0.87, 0.15, 0.12, 0.33]  # [left, bottom, width, height]
ax.set_position(new_position)

# fig.subplots_adjust(right=1)  # Adjust the right margin
fig.suptitle(f"{place_name} Hourly Population ({year:04}-{month:02}-{day:02})", fontsize=20)


# Add colorbar axes at the bottom and align it with the left and right of the subplots
# pos1 = axs[0, 0].get_position() # get the original position for first axis
# pos2 = axs[-1, -1].get_position()
cax = fig.add_axes([0.07, 0.12, 0.007, 0.78])    # [left, bottom, width, height]  # bottom
value_max =vmax
value_min = 0

sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=value_min, vmax=value_max))
sm._A = []
# plt.colorbar(sm, cax=cax, label="Bias")
cbar = plt.colorbar(sm, cax=cax, orientation='vertical')

for t in cbar.ax.get_xticklabels():
    t.set_fontsize(16)

cbar.set_label('Population', labelpad=-60,   rotation=90, fontsize=14, loc='center')

png_name = os.path.join(save_path, f"{place_name.replace(', ', '_')}_hourly_population_{year}{month:02}{day:02}_v4.png")

print("PNG name:", png_name)

plt.savefig(png_name, dpi=300, bbox_inches='tight', pad_inches=0.2)

In [ ]:
merged_gdf['geometry'].explore()

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize = (25, 5))
sum_series = df.iloc[:, 8:].sum()

ACS_popu = sum_series.iloc[0]
sum_series = sum_series.mask(sum_series < 0, ACS_popu)

plt.plot(sum_series.iloc[1:])
plt.axhline(y =ACS_popu, color = 'r', linestyle = '-', lw=0.3) 

hourly_index = pd.date_range(start='2022-01-01', end='2022-12-31 23:00:00', freq='H')
for start in hourly_index[hourly_index.weekday >= 5]:
    plt.axvspan(start, start + pd.Timedelta(days=1), color='lightgrey', alpha=0.1)
    
sum_series

In [ ]:
sum_series.iloc[1:].rolling(window=24, min_periods=1).mean()

In [ ]:


# Given date
date_str = "2024-07-21"
date_obj = datetime.datetime.strptime(date_str, "%Y-%m-%d")

# Get the weekday name
weekday_name = calendar.day_name[date_obj.weekday()]

print(f"The date {date_str} is a {weekday_name}.")


In [ ]:
STOP

# Tools

# Creating the mapping between Places and Census block groups

In [ ]:
CBG_boundary_2019 = r'E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_data\Safegraph_bias\blockgoups\blockgoups2019.shp'
place_boundary_2019 = r"E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\Wild_fire\vectors\cb_2023_us_place_500k.zip"

In [ ]:
CBG_gdf = gpd.read_file(CBG_boundary_2019)
CBG_gdf

In [ ]:
place_gdf = gpd.read_file(place_boundary_2019)
place_gdf

In [ ]:
CBG_gdf['area_deg'] = CBG_gdf.area
CBG_gdf

In [ ]:
inter_gdf = CBG_gdf.overlay(place_gdf, how='intersection')
inter_gdf

In [ ]:
inter_gdf['sub_area_deg'] = inter_gdf.area
inter_gdf['sub_area_ratio'] = inter_gdf['sub_area_deg'] / inter_gdf['area_deg']

inter_gdf

In [ ]:
inter_gdf.query('sub_area_ratio > 0.5')

In [ ]:
saved_fname = r'E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\Wild_fire\vectors\CBG_place.gpkg'
inter_gdf[['sub_area_ratio', 'GEOID_1', 'GEOID_2', 'NAME', 'NAMELSAD', 'STUSPS', 'STATE_NAME', 'geometry']].rename(columns={"GEOID_1":"CBG", "GEOID_2":"place"}).query('sub_area_ratio > 0.5').to_file(saved_fname, driver="GPKG")